In [1]:
from sympy import *
from scipy.integrate import quad
x = symbols('x')
import math

In [4]:
def imcomp_betas(j0_prime, jxbar_prime, theta, sigmastar):
    def tempFunction(i, theta, sigmastar):
        return (i ** ((1 + theta) / theta - 1) * (1 - i) ** ((theta - sigmastar) / theta - 1))

    Bfunvec1_prime = quad(tempFunction, 0, j0_prime, args=(theta, sigmastar))[0]
    Bfunvec2_prime = quad(tempFunction, 0, jxbar_prime, args=(theta, sigmastar))[0]

    return (Bfunvec1_prime, Bfunvec2_prime)

In [5]:
Cex = 0.421601
Ceystar = 26.059874
Cey = 4.598402945
Cem = 1.196110986
jxbar = Cex/(Cex + Ceystar)
jmbar = Cey / (Cey + Cem)
alpha = 0.85
theta = 4
## phi = 5
pe = 0.965289922
te = 0.368354551
tb = 0.368354551
sigmastar = 1
sigma = 1
epsilonS = 0.5  #Homes's energy supply elasticity: beta/(1 - beta)
epsilonSstar = 0.5  #Foreign's energy supply elasticity: betastar/(1 - betastar)
epsilonD = alpha + (1 - alpha) * sigma  #Home's elasticity of demand for embodied energy
epsilonDstar = alpha + (1 - alpha) * sigmastar  #Foreign's elasticity of demand for embodied energy
varphi = 1
prop = 0.449691508

Vg_prime = 38.63009333
Vgstar_prime = 176.5431669

In [6]:
def g(p, alpha = 0.15):
    return alpha**(-alpha) * (1-alpha)**(-(1-alpha)) * p**alpha

def gprime(p, alpha = 0.15):
    return (alpha/(1-alpha))**(1-alpha) * p**(-(1-alpha))

def k(p):
    return gprime(p) / (g(p)-p*gprime(p))

def gdprime(p, alpha = 0.15):
    return -alpha * p**(alpha - 2) * ((1-alpha)/alpha)**alpha

def Dstar(p, sigmastar):
    return gprime(p) * g(p)**(-sigmastar)

def Dstarprime(p, sigmastar):
    return diff(Dstar(x, sigmastar),x).subs(x,p)

def D1star(p,t,sigmastar):
    return g(p)**(-sigmastar) * gprime(p + t)

def D1starprime(p,t,sigmastar):
    return diff(D1star(x,t,sigmastar),x).subs(x,p)

In [7]:
def jxbar_new(pe, tb, jxbar, theta, Cex, Ceystar):
    num = g(pe + tb)**(-theta) * Cex
    denum = g(pe+tb)**(-theta) * Cex + (g(pe) + tb * gprime(pe))**(-theta) * Ceystar
    return num / denum

def j0_new(pe,tb,jxbar,theta,Cex,Ceystar):
    num = g(pe + tb)**(-theta) * Cex
    denum = g(pe+tb)**(-theta) * Cex + (g(pe))**(-theta) * Ceystar
    return num / denum

def Ceystar_new(pe,tb, jxbar_p, jxbar, sigmastar, theta, Ceystar):
    const = gprime(pe) * g(pe)**(-sigmastar) * Ceystar
    frac = ((1-jxbar_p)/ (1-jxbar))**(1+(1-sigmastar)/theta)
    #frac = ((1-jxbar_new(pe,tb,jxbar,theta,Cex,Ceystar))/ (1-jxbar))**(1+(1-sigmastar)/theta)
    return const * frac / (g(1)**(-sigmastar) * gprime(1))

def Cey_new(pe,tb, sigmastar, theta, Cey):
    const = gprime(pe + tb) * g(pe+tb)**(-sigma) * Cey
    return const / (g(1)**(-sigma) * gprime(1))

def Cex1_new(pe,tb, j0bar_p, j0bar, sigmastar, Cex):
    const = gprime(pe + tb) * g(pe+tb)**(-sigmastar) / (g(1)**(-sigmastar) * gprime(1)) * Cex
    frac = (j0bar_p / j0bar) ** (1+ (1-sigmastar)/theta)
    return const * frac

def Cex2_new(pe,tb,jxbar_p, j0bar_p, jxbar, sigmastar, theta):
    const = g(pe)**(-sigmastar) * gprime(pe + tb) / (g(1)**(-sigmastar) * gprime(1)) * Cex
    frac = ((1-jxbar)/jxbar)**(sigmastar/theta) * (theta + 1 - sigmastar)/theta
    jterm = 1/ jxbar**(1+(1-sigmastar)/theta)
    B1, B2 = imcomp_betas(j0bar_p, jxbar_p, theta, sigmastar)
    return const * frac * jterm * (B2- B1)
    

def Cem_new(pe, tb, sigma, Cem):
    const = gprime(pe + tb) * g(pe + tb)**(-sigma) / (g(1)**(-sigmastar) * gprime(1)) * Cem
    return const
    
def Vgx2_new(pe, jxbar_p, jxbar, j0bar_p, Cex, theta, sigmastar):
    pterm = g(pe)**(1-sigmastar) *g(1)* Cex / (gprime(1) * g(1)**(1-sigmastar))
    num = (1-j0bar_p)**((theta + 1 - sigmastar)/theta) - (1-jxbar_p)**((theta + 1 - sigmastar) / theta)
    denum = jxbar * (1-jxbar)**((1-sigmastar)/theta)
    return pterm * num / denum

def Vgx1_new(pe, tb, j0_p, jxbar, Cex, theta, sigmastar):
    const = (g(pe+tb) / g(1))**(1-sigmastar) * (j0_p / jxbar)**(1+(1-sigmastar)/theta) * g(1)/gprime(1) * Cex
    return const

def S_new(pe, tb, Cex2_prime, Vgx2_prime):
    const = g(pe+tb) / gprime(pe+tb) * Cex2_prime - Vgx2_prime
    return const

def Vgm_new(pe, tb, Cem):
    return g(pe+tb)**(1-sigma) * Cem * g(1) / (g(1)**(1-sigma) * gprime(1))

def Lg_new(pe, tb, Cey_prime, Cex1_prime, Cex2_prime):
    return (1/k(pe+tb)) * (Cey_prime + Cex1_prime + Cex2_prime)

def Lgstar_new(pe,tb,Ceystar_prime, Cem_prime):
    return 1/k(pe) * Ceystar_prime + 1/k(pe+tb) * Cem_prime

def Vgstar_new(pe,jxbar_prime,jxbar, sigmastar,theta,Ceystar, Vgx1_prime, Vgx2_prime):
    Vgystar_prime = (g(pe) / g(1))**(1-sigmastar) * ((1-jxbar_prime)/(1-jxbar)) ** (1+(1-sigmastar)/theta) * g(1)/ gprime(1) * Ceystar
    return Vgystar_prime + Vgx1_prime + Vgx2_prime


In [8]:
jxbar_prime = jxbar_new(pe, tb, jxbar, theta, Cex, Ceystar)
jxbar_prime = jxbar
j0_prime = j0_new(pe,tb,jxbar,theta,Cex,Ceystar)
Cey_prime = Cey_new(pe,tb, sigmastar, theta, Cey)
Cex1_prime = Cex1_new(pe,tb, j0_prime, jxbar, sigmastar, Cex)
Cex2_prime = Cex2_new(pe, tb, jxbar_prime, j0_prime, jxbar, sigmastar, theta)
Cem_prime = Cem_new(pe, tb, sigma, Cem)
Ceystar_prime = Ceystar_new(pe,tb, jxbar_prime, jxbar, sigmastar, theta, Ceystar)
Vgx1_prime = Vgx1_new(pe, tb, j0_prime, jxbar, Cex, theta, sigmastar)
Vgx2_prime = Vgx2_new(pe, jxbar_prime, jxbar, j0_prime, Cex, theta, sigmastar)
Vgm_prime = Vgm_new(pe, tb, Cem)
S = S_new(pe, tb, Cex2_prime, Vgx2_prime)
Lg_prime = Lg_new(pe, tb, Cey_prime, Cex1_prime, Cex2_prime)
Lgstar_prime =Lgstar_new(pe,tb,Ceystar_prime, Cem_prime)
Vgstar_prime = Vgstar_new(pe,jxbar_prime,jxbar, sigmastar,theta,Ceystar, Vgx1_prime, Vgx2_prime)

In [9]:
Cex_prime = gprime(pe) * g(pe)**(-sigmastar) / (g(1)**(-sigmastar) * gprime(1)) * (jxbar_prime / jxbar) ** (1+ (1-sigmastar)/theta) * Cex
Ce_prime = Cey_prime + Cem_prime
Cestar_prime = Ceystar_prime + Cex_prime
Ceworld_prime = Ce_prime + Cestar_prime
Qe_prime =4.40505495
Qestar_prime = 27.33067971
Qeworld_prime = Qe_prime + Qestar_prime


In [10]:
dcestardpe = abs(Dstarprime(pe,sigmastar) / Dstar(pe,sigmastar) * Cex_prime 
                      #+ D1starprime(pe,tb,sigmastar) / D1star(pe,tb,sigmastar) * Cex2_prime
                      + Dstarprime(pe,sigmastar) / Dstar(pe,sigmastar) * (Ceystar_prime))
numerator = varphi * (epsilonS * Qe_prime + epsilonSstar * Qestar_prime)
denominator = numerator / varphi + dcestardpe * pe

In [11]:
epsilonSw = (Qe_prime) * epsilonS / Qeworld_prime + Qestar_prime * epsilonSstar / Qeworld_prime
numerator = varphi * epsilonSw * Qeworld_prime
denominator = epsilonSw * Qeworld_prime + epsilonDstar * Cestar_prime

In [194]:
rho = 0.0001

In [208]:
def g1(p, rho = rho):
    alpha = 0.15
    t1 = (1-alpha)**(1/(1-rho))
    t2 = alpha**(1/(1-rho)) * p**(-rho/(1-rho))
    return (t1 + t2)**(-(1-rho)/rho)

def g1prime(p, rho = rho):
    alpha = 0.15
    return diff(g1(x, rho),x).subs(x,p)

In [69]:
pe = 0.977277355
tb = 0.287909396
re = (pe + tb) / pe
ve = (pe + tb)
jmbar_prime =0.767046309
jxbar_prime = 0.013666881
Cey_prime = 3.513027135
Cex_prime = 0.286058256
Cem_prime = 1.381238822
Ceystar_prime = 26.72686083

In [67]:
djxdve = -jxbar_prime * (1-jxbar_prime) * gprime(ve) / g(ve) * theta
djmdve = -jmbar_prime * (1-jmbar_prime) * gprime(ve) / g(ve) * theta

dceydve = Dstarprime(ve, sigma) / Dstar(ve, sigma) * Cey_prime + (1+(1-sigma)/theta) * djmdve / jmbar_prime * Cey_prime
dcemdve = 1/(1-jmbar_prime) * (1+(1-sigma)/theta) * (-djmdve) * Cem_prime 
dcexdve = Dstarprime(ve, sigmastar) / Dstar(ve, sigmastar) * Cex_prime +  (1+(1-sigmastar) / theta) * Cex_prime / jxbar_prime * djxdve
dceystardve = (1+(1-sigmastar) /theta) * Ceystar_prime * (-djxdve) / (1-jxbar_prime)
-(dceystardve + dcemdve) / (dcexdve + dceydve)

0.191695861024254

In [56]:
tejxj = (1 + (1 - sigmastar) / theta) * Cex * (jxbar_prime / jxbar) ** ((1 - sigmastar) / theta) * (pe * re) ** (-epsilonDstar) / jxbar
ejyj = (1 + (1 - sigmastar) / theta) * Cey * (jmbar_prime / jmbar) ** ((1 - sigma) / theta) * (pe * re) ** (-epsilonD) / jmbar
djxdre = -(theta * (1 - alpha)) * jxbar_prime * (1 - jxbar_prime) / re
djmdre = -(theta * (1 - alpha)) * jmbar_prime * (1 - jmbar_prime) / re

leaknum = re * (ejyj * djmdre + tejxj * djxdre)
leakdenum = ejyj * djmdre + tejxj * djxdre - epsilonD * Cey_prime / re - epsilonDstar * Cex_prime / re
leaknum / leakdenum

0.19337472500508746

In [70]:
Cey * (g(pe) / g(ve))**theta / (Cey * (g(pe) / g(ve)) ** theta + Cem)

0.7670469166776344

In [71]:
Cex * (g(pe) / g(ve))**theta / (Cex * (g(pe) / g(ve)) ** theta + Ceystar)

0.013666925821491656